<a href="https://colab.research.google.com/github/dt-cs/Qwen_SFT/blob/main/Qwen3_14b_with_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install fsspec==2023.9.2
    !pip install -U transformers trl
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth


In [ ]:
from datasets import load_dataset
#dataset = load_dataset("deebak14/rhinoscript_ft_data_04", split = "train")

train_dataset = load_dataset("deebak14/rhinoscript_ft_data_04", split = "train")
eval_dataset = load_dataset("deebak14/rhinoscript_ft_data_04_test", split = "train")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "Qwen/Qwen3-14B"            #"Qwen/Qwen2.5-Coder-7B-Instruct" #"mistralai/Mistral-7B-Instruct-v0.3" #"google/gemma-3-12b-it"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Setup bitsandbytes 4-bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Load the model in 4bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",         # auto-assigns model to GPU if available
    torch_dtype=torch.float16, # or 'auto' if you want auto-detection
)

# Set max sequence length when tokenizing/generating, not in .from_pretrained
max_seq_length = 4096


In [4]:
from peft import LoraConfig, get_peft_model

# Configure LoRA
lora_config = LoraConfig(
    r=4,                     # LoRA rank
    lora_alpha=8,           # LoRA alpha (often 2x r)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.01,
    bias="none",
    task_type="CAUSAL_LM",    # Important for language modeling
)

# Attach LoRA to model
model = get_peft_model(model, lora_config)

# (Optional) Print trainable parameters to confirm LoRA is applied
model.print_trainable_parameters()

trainable params: 16,056,320 || all params: 14,784,363,520 || trainable%: 0.1086


In [5]:
with open("/content/qwen_chat_template.jinja") as f:
    tokenizer.chat_template = f.read()

In [ ]:
from trl import apply_chat_template

# Pick a few examples from your dataset
for i in range(3):
    sample = train_dataset[i]
    # Apply the chat template with tokenization and assistant mask
    output = tokenizer.apply_chat_template(
        sample['messages'],
        tokenize=True,
        return_assistant_tokens_mask=True,
        return_dict=True,
    )

    print(f"Sample {i}:")
    print("".join(map(str, output["assistant_masks"])))
    print("-" * 40)

In [ ]:
from trl import SFTTrainer, SFTConfig

sft_config = SFTConfig(
    dataset_text_field="messages",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_ratio=0.05,                # Increased for more stable start
    num_train_epochs=10,
    learning_rate= 2e-6,             #1e-5             # Lowered for better generalization
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.05,
    lr_scheduler_type="cosine",
    seed=3407,
    report_to="none",
    eval_strategy="epoch",          # Evaluate and save every epoch
    save_strategy="epoch",
    save_total_limit=2,             # Only keep last 3 checkpoints
    max_grad_norm=1.0,              # Clip gradients
    fp16=True,                      # Enable mixed-precision if available (optional, can remove if not supported)
    push_to_hub=False,
    neftune_noise_alpha=5,
    assistant_only_loss=True,
    chat_template_path="/content/qwen_chat_template.jinja"
)

trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)


In [ ]:
# Get the full input_ids and assistant_masks for a sample
sample = trainer.train_dataset[100]
input_ids = sample["input_ids"]
assistant_masks = sample["assistant_masks"]

# Extract token ids for assistant tokens only
assistant_token_ids = [tid for tid, mask in zip(input_ids, assistant_masks) if mask == 1]

# Now decode ONLY those assistant tokens
assistant_text = tokenizer.decode(assistant_token_ids)
print("Assistant text only:\n", assistant_text)


In [9]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
13.855 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [10]:
trainer_stats = trainer.train()

Epoch,Training Loss,Validation Loss
1,2.357200,2.056313
2,1.631800,1.410493
3,1.188800,0.995862
4,1.033100,0.898313
5,0.884500,0.877809
6,0.845500,0.871156
7,0.863700,0.867139
8,0.855400,0.865297
9,0.853500,0.864410
10,0.807400,0.864205


In [11]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3875.2585 seconds used for training.
64.59 minutes used for training.
Peak reserved memory = 32.387 GB.
Peak reserved memory for training = 18.532 GB.
Peak reserved memory % of max memory = 81.874 %.
Peak reserved memory for training % of max memory = 46.849 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Qwen-3` team, the recommended settings for reasoning inference are `temperature = 0.6, top_p = 0.95, top_k = 20`

For normal chat based inference, `temperature = 0.7, top_p = 0.8, top_k = 20`

# Before SFT

In [ ]:
from transformers import pipeline

# Use a pipeline for easy inference
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example usage
messages = [
    {"role" : "system",
     "content" : """You are an expert in Python and Rhino 3D modeling using the rhinoscriptsyntax module and, if required, other native Python modules to accomplish the task. Your task is to interpret, analyze, and understand the user’s query, then respond with a corresponding Python script. Whenever you use a method from rhinoscriptsyntax, always ensure that it is an official part of the module, and that you use the correct number and types of arguments for each function. Your final code output should precisely capture the user’s intent, be free of ambiguity, and run without errors.\n\nOutput should be a python code inside ```python ``` block. """},
    {"role" : "user",
     "content" : "how to create a box."}
]

prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking = True
)

outputs = pipe(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
        top_k=20,
        min_p=0
)
print(outputs[0]["generated_text"])

# After SFT

In [ ]:
from transformers import pipeline

# Use a pipeline for easy inference
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example usage
messages = [
    {"role" : "system",
     "content" : """You are an expert in Python and scripting using the rhinoscriptsyntax module. Your task is to understand user query and generate a script accordingly to model in rhino 3d.
     Please reason step by step. Breakdown the user query into parts, based on which write an algorithm, then write the code. Do not prompt user for input in the code unless specified.
     Your Output should be a python code inside ```python ``` block. """},
    {"role" : "user",
     "content" : "create boxes along circle."}
]

prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking = True
)

outputs = pipe(
        prompt,
        max_new_tokens=2048,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
        top_k=20,
        min_p=0
)
print(outputs[0]["generated_text"])